<a href="https://colab.research.google.com/github/vardhan1379/ARYA.ai-assignment/blob/main/ARYA_AI_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Exploratory Data Analysis**

---
**This dataset contains 57 features named from X1 to x57 and a label column named Y. While observing the discreteness in the label's column, we can consider its an classification problem.While observing the data I can conclude that there are lot of zeros in the data,which isn't a proper distribution to frame a model i.e.,This model might have an tendency to zero inflation. To avoid this, standardization of data is done using standard scaler. In general, feature selection is crucial step in data engineering although there are no specified feature names in this data we perform chi-squared test to determine the 35 most important features while determining Y( feature selection is used to avoid the curse of dimensionality and to maintain Occam's razor).




# **Import Required Libraries**

In [ ]:
import numpy as np
from numpy import array 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from numpy import asarray
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier,BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from xgboost import XGBClassifier,XGBRFClassifier

# **Read Data**

In [ ]:
Data = pd.read_csv("training_set.csv",delimiter=",",index_col=0)
print(Data)
Test_data = pd.read_csv("test_set.csv",delimiter=",",index_col=0)
print(Test_data)

**Get information of the Data** 

In [ ]:
Data.describe()

**while watching the above results we can conclude that" There are outliers in our data" because there is massive difference b/w 75 th percentile and max values.**

---

Note: I have tried to include various plots such as t-sne, PCA plots and Box plots. Due to high number of features there is problem of overlapping of features in the plots. So I've decided to remove them and tried to understand the Data based on Corelation Heat map and Data dispersion quatifiers.

In [ ]:
Data.info()

**We can see that there are no NULL values in any of the attributes and it only consists of integers and float values.**

---
**Now we will plot the correlation heat map for the data**


In [ ]:
plt.figure(figsize=(100,100))
sns.heatmap(Data.iloc[:,:-1].corr(),cmap='viridis',annot=True) 

**We can observe there are few highly positive correlated and Few Highly Negative correlated variables, Using Chi-squared test we can remove Multicollinearity**

---



**Lets Check for imbalance data**

In [ ]:
target_count = Data.Y.value_counts()
print('Class 0:', target_count[0])
print('Class 1:', target_count[1])
print('Proportion:', round(target_count[0] / target_count[1], 2), ': 1')

target_count.plot(kind='bar', title='Count (target)');

#**Preprocess the Data**

**Now we use data for feature selection step. This feature selcton step avoids multi-collinearity between the features while improving the performance of model **

In [ ]:
select = SelectKBest(score_func=chi2, k=35)
z = select.fit_transform(Data.iloc[:,:-1],Data.iloc[:,-1])
feature_names =  select.get_feature_names_out(Data.iloc[:,:-1].columns)
print("Important faeture names are",feature_names)
print("Shape of Train data after Feature selection:", z.shape)
Data_sel = Data[feature_names]
print(Data_sel.shape)
#Now remove the less important features in test data too
Test_data_sel = Test_data[feature_names]
print("Shape of test data after feature selection is",Test_data_sel.shape)

**Standardization using standard scaler**

In [ ]:
scaler = StandardScaler()
#Transform data
Data_scaled = scaler.fit_transform(Data_sel)
print("Scaled Data shape:",Data_scaled.shape)
print(Data_sel)
#sacling the test data
Test_data_scaled = scaler.fit_transform(Test_data_sel)
print("Scaled test data shape:",Test_data_scaled.shape)
print(Test_data_scaled)

**Now split the "Data" into training set and validation set**

In [ ]:
#Now we ready the whole Data into Train, test and validation data
X_train, X_val, y_train, y_val = train_test_split(Data_scaled, Data.iloc[:,-1], test_size=0.2, random_state=1)
Testing_data = Test_data_scaled
print("Training_data shape:",X_train.shape,y_train.shape)
print(" Validation_data shape:",X_val.shape,y_val.shape)
print("Test _data shape:",Testing_data.shape)


**Training and Validation**

---
**1. Models that are used to check the individual performance**


In [ ]:
models = []
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))
models.append(('KNN-1', KNeighborsClassifier(n_neighbors=1)))
models.append(('KNN-3', KNeighborsClassifier(n_neighbors=3)))
models.append(('KNN-5', KNeighborsClassifier(n_neighbors=5)))
models.append(('KNN-7', KNeighborsClassifier(n_neighbors=7)))
models.append(('KNN-9', KNeighborsClassifier(n_neighbors=9)))
models.append(('KNN-11', KNeighborsClassifier(n_neighbors=11)))
models.append(('KNN-13', KNeighborsClassifier(n_neighbors=13)))
models.append(('AB', AdaBoostClassifier()))
models.append(('GBM', GradientBoostingClassifier()))
models.append(('RF', RandomForestClassifier()))
models.append(('ET', ExtraTreesClassifier()))
models.append(("BC",BaggingClassifier()))
models.append(("XGBC",XGBClassifier()))



**2. Performance analysis metrics**

In [ ]:
def plotconfusion_matrix(modelname,y_test,y_pred):
   
    conf_mat = confusion_matrix(y_true=y_test, y_pred=y_pred)
    print(modelname+'Confusion matrix:\n', conf_mat)
    labels = ['Class 0', 'Class 1']
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(conf_mat, cmap=plt.cm.Blues)
    fig.colorbar(cax)
    ax.set_xticklabels([''] + labels)
    ax.set_yticklabels([''] + labels)
    plt.xlabel('Predicted')
    plt.ylabel('Expected')
    plt.show()
model_lis= []
def modelfitpredict(modelname,model,X_train, X_test, y_train, y_test):
    
    
    model = model
    model.fit(X_train,y_train)
    model_lis.append(model)
    y_pred = model.predict(X_test)
    plotconfusion_matrix(name,y_test,y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    print(name+" Accuracy: %.2f%%" % (accuracy * 100.0))
    auc = roc_auc_score(y_test, y_pred)
    print(name+' ROC AUC: %f' % auc)
    print("-"*50)
    #results.update( {name : accuracy*100.0} )
    return name, accuracy, model_lis

**3.Training and performance evaluation of each models**

In [ ]:
results = {}
for name, model in models:
    name_, accuracy, model_s = modelfitpredict(name,model,X_train, X_val, y_train, y_val)
    results[name_] = accuracy*100.0
    
print("Validation results on test data :",results)

**4.Accuarcy comparision of various models**

In [ ]:
print(results)
plt.bar(*zip(*results.items()))
plt.show()

**The highest scoring algorithms are RandomForest and ETree classifiers which they have attained an acuarcy of 95.26%. Also we can see the accuracies of various classifiers is merely ditinguishable, But the performance of Decision Trees, boosting and bagging classifiers are the highest (They have the advantage of making multiple decisions and producing their decision as collective decision makes them high scorers in this sceneraio). Although, having advantageous classifiers in competition, Logistic Regression classifier performed exceptionally well for such an simple algorithm.(It performed well because of the step "Feature Selection". This Feature Selection helped the model to retain unnecessary variables in equation which in then helped the logistic regression model to achieve the accuarcy of 91.94%).**





---

## **Now we will use the RandomForest classifier to predict the Test Set**

In [ ]:
model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(Testing_data)
print(y_pred)

**Now above values are the predicted values for test dataset**